In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
#-------------------------------------------------gray--------------------------------------------------
input = r"D:\Visual Disorders\PreProcess\Original\4"
i = 0
kernel = np.ones((10,10),np.uint8)
for im in os.listdir(input):  
    path = os.path.join(input,im)
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)[1]
    
    cv2.imwrite(os.path.join(r"D:\Visual Disorders\PreProcess\Otsu\4","OtsuThresholding" +str(i)+'.png'),thresh)
    
    

    # Find contour and sort by contour area
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)


    # Find bounding box and extract ROI
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        ROI = image[y:y+h, x:x+w]
        break
    cv2.imwrite(os.path.join(r"D:\Visual Disorders\PreProcess\Crop(ROI)\4","ROI" +str(i)+'.png'),ROI)
    ret,thresh1 = cv2.threshold(ROI,0,255,cv2.THRESH_BINARY)
    cv2.imwrite(os.path.join(r"D:\Visual Disorders\PreProcess\BinaryMaskingBeforeOpening\4","BinaryMasking" +str(i)+'.png'),thresh1)
    mask = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel)
    cv2.imwrite(os.path.join(r"D:\Visual Disorders\PreProcess\Mask_Open\4","Mask" +str(i)+'.png'),mask)
    result = cv2.bitwise_and(ROI, mask)
    cv2.imwrite(os.path.join(r"D:\Visual Disorders\PreProcess\Mask+ROI\4","Result" +str(i)+'.png'),result)
    
      #NonLocal Means Denoising (NLMD)    
    img = cv2.fastNlMeansDenoisingColored(result, None, 1, 1, 7, 21)
    cv2.imwrite(os.path.join(r"D:\Visual Disorders\PreProcess\NonLocal Means Denoising\4","NLMD" +str(i)+'.png'),img)
    
    
    
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (8, 8))
    img_yuv[:,:,0] = clahe.apply(img_yuv[:,:,0])
    img = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    cv2.imwrite(os.path.join(r"D:\Visual Disorders\PreProcess\Clahe\4","Clahe" +str(i)+'.png'),img)
    image = cv2.resize(img, (512,512))
    cv2.imwrite(os.path.join(r"D:\Visual Disorders\PreProcess\FinalResize\4","FinalResize" +str(i)+'.png'),image)
    i += 1
  